In [1]:
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
# from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.indexes import VectorstoreIndexCreator
from langchain.chat_models import openai
from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI
import requests
import os
from datetime import datetime
from time import sleep
import pandas as pd
import faiss
from pathlib import Path

In [2]:
os.environ["OPENAI_KEY"] = "b98951878a904abb8ed21a264523758c"
os.environ["AZURE_OPENAI_KEY"] = os.environ["OPENAI_KEY"]
os.environ["AZURE_OPENAI_API_KEY"] = os.environ["OPENAI_KEY"]
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://lab-ai-openai-eus.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo"

In [3]:
client = AzureOpenAI(
  azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-01" #2024-02-15-preview ?
)

message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."}]

completion = client.chat.completions.create(
  model="gpt-35-turbo", # model = "deployment_name"
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

print(completion)

ChatCompletion(id='chatcmpl-9XR6Sf5s3JRY8vLuGInVO7xM6a0vn', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Great! How can I assist you today?', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1717756036, model='gpt-35-turbo', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=19, total_tokens=28), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])


In [4]:
llm = AzureChatOpenAI(
    openai_api_version="2024-02-01",
    azure_deployment="gpt-35-turbo",
)

llm4 = AzureChatOpenAI(
    openai_api_version="2024-05-13",
    azure_deployment="gpt-4o",
)

from langchain_core.messages import HumanMessage

message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
llm.invoke([message])

AIMessage(content="J'aime programmer.", response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 20, 'total_tokens': 25}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-84af2c29-14e3-42d2-8d50-08fb6c2f1c5a-0', usage_metadata={'input_tokens': 20, 'output_tokens': 5, 'total_tokens': 25})

In [5]:
# !pip install unstructured
# !pip install "unstructured[pdf]"

In [6]:
from langchain_community.document_loaders import DirectoryLoader


directory_path = "microsoft_learn"

loader = DirectoryLoader(directory_path)
pages = loader.load_and_split()
pages

# partition_pdf is not available. Install the pdf dependencies with pip install "unstructured[pdf]"

[Document(page_content="Attention !\n\nCe fichier contient des informations confidentielles et sensibles qui ne doivent pas être divulguées à des tiers. La lecture de ce fichier est strictement réservée aux personnes autorisées par le propriétaire ou le créateur du fichier. Toute tentative de lecture non autorisée ou de copie du fichier constitue une violation de la loi et expose le contrevenant à des sanctions pénales et civiles.\n\nSi vous n'êtes pas une personne autorisée, veuillez fermer immédiatement ce fichier et le supprimer de votre appareil. Si vous avez reçu ce fichier par erreur, veuillez en informer le propriétaire ou le créateur du fichier et lui renvoyer ou lui détruire le fichier. Si vous êtes une personne autorisée, veuillez vous assurer que le fichier est protégé par un mot de passe et qu'il n'est pas accessible à des personnes non autorisées.\n\nMerci de respecter la confidentialité et la sécurité des informations contenues dans ce fichier.", metadata={'source': 'micr

RAG : recherche par similarité (en utilisant les embeddings)

Extract text from pdf 

In [7]:
#!pip install pdfminer.six

In [ ]:
# from pdfminer.high_level import extract_text


# dirs = os.listdir(directory_path)
# output = "pdftotext.txt"
# # supprimer le fichier s'il existe déjà
# if os.path.exists(output):  
#     os.remove(output) 
# for file in dirs:
#     path_file = os.path.join(directory_path, file)
#     print(f"PDF path file: {path_file}")
#     text = extract_text(path_file)
#     with open(output, 'a', encoding='utf-8') as fichier:
#         fichier.write(text)


In [ ]:
# !pip install pypdf

In [9]:
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader


# loader = TextLoader("pdftotext.txt", encoding='utf-8')
# loader = PyPDFLoader("microsoft_learn/intelligence_artificielle.pdf")
loader = DirectoryLoader("microsoft_learn/", glob="**/*.pdf", loader_cls=PyPDFLoader)

In [10]:
# loader multi files types
# https://github.com/langchain-ai/langchain/discussions/18559

In [11]:
documents= loader.load()
for document in documents:
    document.metadata["filename"] = Path(document.metadata["source"]).name
    document.metadata["loading_date"] = datetime.today().strftime('%Y%m%d')
    document.metadata["user"] = Path(document.metadata["source"]).parent.name
len(documents)

51

In [12]:
documents[0].metadata

{'source': 'microsoft_learn\\Liticia\\Attention.pdf',
 'page': 0,
 'filename': 'Attention.pdf',
 'loading_date': '20240607',
 'user': 'Liticia'}

In [13]:
documents[1].metadata

{'source': 'microsoft_learn\\Paul\\Le deep learning pour les images.pdf',
 'page': 0,
 'filename': 'Le deep learning pour les images.pdf',
 'loading_date': '20240607',
 'user': 'Paul'}

In [14]:
documents[0].page_content

"Attention !  \nCe fichier contient des informations confidentielles et sensibles qui ne doivent pas être divulguées \nà des tiers. La lecture de ce fichier est strictement réservée aux personnes autorisées par le \npropriétaire ou le créateur du fichier. Toute tentative de l ecture non autorisée ou de copie du fichier \nconstitue une violation de la loi et expose le contrevenant à des sanctions pénales et civiles.  \nSi vous n'êtes pas une personne autorisée, veuillez fermer immédiatement ce fichier et le \nsupprimer de votre appareil. Si vous avez reçu ce fichier par erreur, veuillez en informer le \npropriétaire ou le créateur du fichier et lui renvoyer ou lui détruire l e fichier. Si vous êtes une \npersonne autorisée, veuillez vous assurer que le fichier est protégé par un mot de passe et qu'il \nn'est pas accessible à des personnes non autorisées.  \nMerci de respecter la confidentialité et la sécurité des informations contenues dans ce fichier.  \n "

In [15]:
# chunk settings  
chunk_size = 200
chunk_overlap = 30 # duplication des informations entre les vecteurs coupés 

In [16]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter


# text_splitter = CharacterTextSplitter(
#                     chunk_size=chunk_size,
#                     chunk_overlap=chunk_overlap,
#                     separator="\n",
#                 )

text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=chunk_size,
                    chunk_overlap=chunk_overlap,
                    separators=["\n"],
                )
chunks = text_splitter.split_documents(documents)

nb_chunks = len(chunks)
print(f"Nb of chunks: {nb_chunks}")

Nb of chunks: 1067


In [17]:
chunks[0]

Document(page_content='Attention !  \nCe fichier contient des informations confidentielles et sensibles qui ne doivent pas être divulguées', metadata={'source': 'microsoft_learn\\Liticia\\Attention.pdf', 'page': 0, 'filename': 'Attention.pdf', 'loading_date': '20240607', 'user': 'Liticia'})

In [18]:
print(chunks[nb_chunks-1].metadata["user"])
print(chunks[nb_chunks-1].page_content)

Paul
évolution, qui continue de proposer des modèles et des techniques innovants, pour relever les défis liés 
à la complexité et à la diversité des données visuelles.


In [19]:
# # https://stackoverflow.com/questions/76886954/multiple-file-loading-and-embeddings-with-openai

# print("Loading data...")
# pdf_folder_path = "content/"
# print(os.listdir(pdf_folder_path))

# # Load multiple files
# loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]

# print(loaders)

# all_documents = []

# for loader in loaders:
#     print("Loading raw document..." + loader.file_path)
#     raw_documents = loader.load()

#     print("Splitting text...")
#     text_splitter = CharacterTextSplitter(
#         separator="\n\n",
#         chunk_size=800,
#         chunk_overlap=100,
#         length_function=len,
#     )
#     documents = text_splitter.split_documents(raw_documents)
#     all_documents.extend(documents)

# print("Creating vectorstore...")
# embeddings = OpenAIEmbeddings()
# vectorstore = FAISS.from_documents(all_documents, embeddings)

# with open("vectorstore.pkl", "wb") as f:
#     pickle.dump(vectorstore, f)

Enregistrer les chunks en tant que vecteur en utilisant faiss  
pour résoudre le problème des tokens appels nombreux API, on découpe les chunks
 en blocs , sleep 
save le tout en local pour ne pas relancer à chaque fois

In [20]:
# !pip install -U langchain-community faiss-cpu langchain-openai tiktoken

In [21]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import AzureOpenAIEmbeddings
# https://github.com/facebookresearch/faiss/wiki/Special-operations-on-indexes

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    api_key=os.environ["AZURE_OPENAI_KEY"],
)

CALC_EMBEDDINGS = False
BATCH_EMBEDDINGS = False

if CALC_EMBEDDINGS:
    if BATCH_EMBEDDINGS:

        # Iterate over each chunk of documents
        # https://github.com/langchain-ai/langchain/issues/7343
        # https://stackoverflow.com/questions/76644262/adding-large-chunks-of-text-after-embedding-into-pinecone-without-openai-ratelim
        batch_size = 50

        for i in range(0, len(chunks), batch_size):
            print(f"batch {i} to {i+batch_size}")
            if 'faiss_vectorstore' not in locals() or 'faiss_vectorstore' not in globals():
                faiss_vectorstore = FAISS.from_documents(chunks[i:i+batch_size], embeddings)
            else:
                faiss_vectorstore.add_documents(chunks[i:i+batch_size])
                print(f"...sleep 6s...")
                sleep(6)  # Add delay here
    else:
        faiss_vectorstore = FAISS.from_documents(chunks, embeddings)

    faiss_retriever = faiss_vectorstore.as_retriever()
    faiss_vectorstore.save_local("faiss_index")
else:
    FAISS.load_local("faiss_index", embeddings)

# TODO : optimiser la taille des chunks plus tard
# LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.

c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [22]:
# import pickle

# #with open("index.pkl", "ab") as f:
# with open("index.pkl", "wb") as f:
#     pickle.dump(faiss_vectorstore, f)
#     f.close()

# # DEBUG : cannot pickle '_thread.RLock' object
# # https://github.com/langchain-ai/langchain/issues/14581

In [23]:
question = "Qu'est ce que le service Azure OpenAI ?"
question = "Citer trois services cognitifs Azure."
# question = "Ce document est-il autorisé ?"

In [24]:
searchDocs = faiss_vectorstore.similarity_search(question, filter={"user":"Paul"} )

print(f"Retrieve docs: {len(searchDocs)}")
for i in range(len(searchDocs)):
    print(f"- {i} from {searchDocs[i].metadata['filename']} : \n'''{searchDocs[i].page_content}'''")

Retrieve docs: 4
- 0 from Les modèles de fondation de l IA générative.pdf : 
'''comprendre et d'analyser le contenu visuel, comme les images, les vidéos ou les flux en direct. Les 
services cognitifs Azure de la vision comprennent :'''
- 1 from Le deep learning pour les images.pdf : 
'''comprendre et d'analyser le contenu visuel, comme les images, les vidéos ou les flux en direct. Les 
services cognitifs Azure de la vision comprennent :'''
- 2 from Les services cognitifs Azure.pdf : 
'''comprendre et d'analyser le contenu visuel, comme les images, les vidéos ou les flux en direct. Les 
services cognitifs Azure de la vision comprennent :'''
- 3 from Le service Azure OpenAI.pdf : 
'''comprendre et d'analyser le contenu visuel, comme les images, les vidéos ou les flux en direct. Les 
services cognitifs Azure de la vision comprennent :'''


Retrievers are used to find relevant documents or passages that contain the answer to a given query.
The system "retrieves" any documents that could be relevant in answering the question, and then passes those documents (along with the original question) to the language model for a "generation" step.

La question est posée en utilisant le chat,
toujours en utilisant la similarité sématique 


Test utilisant LCEL


In [25]:
prompt_template = """Utilisez les éléments de contexte suivants pour répondre à la question contenue dans les 3 crochets à la fin. Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.
Veuillez fournir une réponse correcte sur le plan factuel et basée sur les informations extraites du magasin de vecteurs.
Veuillez également mentionner toute citation étayant la réponse, si elle est présente dans le contexte fourni, entre deux doubles guillemets "" .

{context}

QUESTION:```{question}```
ANSWER:
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context","question"]
  )

chain_type_kwargs = {"prompt": PROMPT}

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


# prompt_template =  """
#                     Answer the question only according to the context: {context}
#                     Question: {question}
#                     """

# PROMPT = ChatPromptTemplate.from_template(prompt_template)

retrieval_chain = (
    {"context": faiss_retriever, "question": RunnablePassthrough()}
    | PROMPT
    | llm
    | StrOutputParser()
)

In [27]:
question = "Qu'est ce que le service Azure OpenAI ?"
question = "Citer trois services cognitifs Azure."

In [28]:
retrieval_chain.invoke(question)

'Le texte donne un seul service cognitif Azure, qui est celui de la vision. Aucun autre service n\'est mentionné. Donc, la réponse à la question est: "Le texte ne donne qu\'un seul service cognitif Azure, qui est celui de la vision."'

In [29]:
from langchain.chains import RetrievalQA


filtered_retriever = faiss_vectorstore.as_retriever(search_kwargs={"filter": {"user":'Paul'}})

qa_stuff = RetrievalQA.from_chain_type( #creer une chain pour répondre à la question
    llm=llm, 
    chain_type="stuff", # tous les documents d'un coup et les passe au LLM
    # chain_type_kwargs=chain_type_kwargs,
    retriever=filtered_retriever, 
    return_source_documents= True,
    verbose=True
)

In [30]:
# question = "Pourquoi utiliser Azure OpenAI ?"
# question = "Quel service utiliser pour de la traduction ?"
question = "Est-ce que le contenu du contexte est autorisé ?"

result = qa_stuff.invoke(question)
result



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Est-ce que le contenu du contexte est autorisé ?',
 'result': "Oui, le contenu du contexte est autorisé. Il s'agit d'une répétition de la même phrase qui décrit la capacité à générer du contenu à partir de différents types de données brutes.",
 'source_documents': [Document(page_content='générer du contenu à partir de données brutes, comme du texte, des images, des sons ou des vidéos.', metadata={'source': 'microsoft_learn\\Paul\\Le deep learning pour les images.pdf', 'page': 0, 'filename': 'Le deep learning pour les images.pdf', 'loading_date': '20240607', 'user': 'Paul'}),
  Document(page_content='générer du contenu à partir de données brutes, comme du texte, des images, des sons ou des vidéos.', metadata={'source': 'microsoft_learn\\Paul\\Le service Azure OpenAI.pdf', 'page': 0, 'filename': 'Le service Azure OpenAI.pdf', 'loading_date': '20240607', 'user': 'Paul'}),
  Document(page_content='générer du contenu à partir de données brutes, comme du texte, des images, des son

Préparer un golden dataset 


In [31]:
# TODO : generate benchmark dataset with RAGAS
# https://docs.ragas.io/en/stable/getstarted/testset_generation.html

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# generator with openai models
generator_llm = llm
critic_llm = llm4
# embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})


c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating:   0%|          | 0/10 [00:00<?, ?it/s]                

In [ ]:
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the API Azure OpenAI and how can they...,[ce que le service Azure OpenAI ? \nLe service...,nan,simple,[{'source': 'pdftotext.txt'}],True
1,What is the purpose of the service for extract...,[ qui permet d'extraire et de structurer les d...,nan,simple,[{'source': 'pdftotext.txt'}],True
2,What is the Model R-CNN and how does it work i...,"[u de \ndonnées de plus petite taille, en cons...",nan,simple,"[{'source': 'pdftotext.txt'}, {'source': 'pdft...",True
3,What are one-stage models and how do they diff...,"[ le \nmodèle R-CNN est très lent, car il néce...",nan,simple,[{'source': 'pdftotext.txt'}],True
4,What are the API Azure OpenAI and how can they...,[ce que le service Azure OpenAI ? \nLe service...,nan,simple,[{'source': 'pdftotext.txt'}],True
5,What are the two categories of CNN-based model...,"[u de \ndonnées de plus petite taille, en cons...",nan,reasoning,[{'source': 'pdftotext.txt'}],True
6,How does Azure OpenAI service utilize AI and w...,[ce que le service Azure OpenAI ? \nLe service...,The Azure OpenAI service utilizes AI by provid...,reasoning,[{'source': 'pdftotext.txt'}],True
7,How do CNNs aid in image generation for Azure ...,[Les modèles de fondation de l’IA générative \...,nan,multi_context,[{'source': 'pdftotext.txt'}],True
8,How has deep learning enhanced object detectio...,"[u de \ndonnées de plus petite taille, en cons...",nan,multi_context,[{'source': 'pdftotext.txt'}],True
9,What is the task of object detection and how h...,"[u de \ndonnées de plus petite taille, en cons...",The task of object detection is to identify an...,simple,[{'source': 'pdftotext.txt'}],True


In [ ]:
print(f"Test size: {testset.to_pandas().size}")

Test size: 60


In [ ]:
questions = [
"Qu'est-ce que le service Azure OpenAI ?",
"Comment fonctionne le service Azure OpenAI ?",
"Quels sont les avantages du service Azure OpenAI ?",
"Quels sont les cas d'utilisation du service Azure OpenAI ?",
"Comment mettre en œuvre le service Azure OpenAI ?",
"Quels sont les composants principaux du service Azure OpenAI ?",
"Quels sont les modèles d'IA accessibles via le service Azure OpenAI ?",
"Quelles tâches les modèles d'IA du service Azure OpenAI peuvent-ils accomplir ?",
"Quelle est l'infrastructure sur laquelle repose le service Azure OpenAI ?",
"Quels outils le service Azure OpenAI fournit-il pour la formation et le réglage des modèles d'IA ?"
]

answers = [  
"Le service Azure OpenAI est un service cloud qui permet aux développeurs et aux chercheurs d'accéder à la plateforme OpenAI, une initiative qui vise à créer une intelligence artificielle générale (AGI) bénéfique pour l'humanité. Il offre la possibilité d'utiliser des modèles d'IA pré-entraînés ou personnalisés, de les déployer sur le cloud ou en périphérie, et de les intégrer à des applications et des scénarios variés.",   
"Le service Azure OpenAI repose sur deux composants principaux : le portail Azure OpenAI et les API Azure OpenAI. Le portail est une interface web qui permet de créer, de gérer et de surveiller les ressources du service, tandis que les API sont des interfaces de programmation qui permettent d'interagir avec les modèles d'IA du service.",   
"Le service Azure OpenAI présente plusieurs avantages, parmi lesquels l'accès à des modèles d'IA de pointe, une grande flexibilité, une haute performance grâce à l'infrastructure cloud de Microsoft Azure, et une facilité d'intégration grâce à des API simples et des SDK dans différents langages de programmation.",   
"Le service Azure OpenAI peut être utilisé pour de nombreux cas d'utilisation, dans différents domaines et secteurs d'activité, tels que l'éducation, la santé, le divertissement et le commerce.",   
"Pour mettre en œuvre le service Azure OpenAI, il faut créer un compte Microsoft Azure, souscrire à un abonnement au service, créer une clé d'API, choisir ou créer un modèle d'IA, créer un point de terminaison, envoyer des requêtes au point de terminaison, et analyser les résultats des requêtes.",   
"Les composants principaux du service Azure OpenAI sont le portail Azure OpenAI et les API Azure OpenAI.",   
"Les modèles d'IA accessibles via le service Azure OpenAI incluent GPT-3, Codex, DALL-E et CLIP.",   
"Les modèles d'IA du service Azure OpenAI peuvent réaliser des tâches complexes et variées, telles que la génération de texte, la compréhension du langage naturel, la vision par ordinateur, la synthèse vocale ou la création de contenu multimédia.",   
"L'infrastructure sur laquelle repose le service Azure OpenAI est l'infrastructure cloud de Microsoft Azure.",   
"Le service Azure OpenAI fournit des outils de formation et de réglage pour créer son propre modèle d'IA."
]  

ground_truths = [  
["Le service Azure OpenAI est un service cloud qui permet aux développeurs et aux chercheurs d'accéder à la plateforme OpenAI", "Le service Azure OpenAI offre la possibilité d'utiliser des modèles d'IA pré-entraînés ou personnalisés, de les déployer sur le cloud ou en périphérie, et de les intégrer à des applications et des scénarios variés."],  
["Le service Azure OpenAI repose sur deux composants principaux : le portail Azure OpenAI et les API Azure OpenAI.", "Le portail Azure OpenAI est une interface web qui permet de créer, de gérer et de surveiller les ressources du service Azure OpenAI, telles que les abonnements, les clés, les modèles, les points de terminaison et les requêtes."],  
["Le service Azure OpenAI présente plusieurs avantages, parmi lesquels :", "Il garantit une haute performance, en s'appuyant sur l'infrastructure cloud de Microsoft Azure, qui assure une disponibilité, une scalabilité et une sécurité optimales."],  
["Le service Azure OpenAI peut être utilisé pour de nombreux cas d'utilisation, dans différents domaines et secteurs d'activité.", "Dans le domaine du commerce, le service Azure OpenAI peut être utilisé pour créer des outils d'optimisation, tels que des recommandations personnalisées, des prévisions de ventes, des analyses de marché ou des chatbots."],  
["Pour mettre en œuvre le service Azure OpenAI, il faut suivre les étapes suivantes :", "Analyser les résultats des requêtes, en utilisant les outils de suivi et de statistiques fournis par le portail Azure OpenAI."],  
["Le service Azure OpenAI repose sur deux composants principaux : le portail Azure OpenAI et les API Azure OpenAI.", "Les API Azure OpenAI sont des interfaces de programmation qui permettent d'interagir avec les modèles d'IA du service Azure OpenAI, tels que GPT-3, Codex, DALL-E ou CLIP."],  
["Les API Azure OpenAI sont des interfaces de programmation qui permettent d'interagir avec les modèles d'IA du service Azure OpenAI", "tels que GPT-3, Codex, DALL-E ou CLIP."],  
["Il permet d'accéder à des modèles d'IA de pointe, capables de réaliser des tâches complexes et variées", "telles que la génération de texte, la compréhension du langage naturel, la vision par ordinateur, la synthèse vocale ou la création de contenu multimédia."],  
["Il garantit une haute performance, en s'appuyant sur l'infrastructure cloud de Microsoft Azure", "qui assure une disponibilité, une scalabilité et une sécurité optimales."],  
["Choisir un modèle d'IA, parmi ceux proposés par le service Azure OpenAI", "ou créer son propre modèle, en utilisant les outils de formation et de réglage fournis par le service Azure OpenAI."]  
]

results = []

contexts = []

In [ ]:
for query in questions:
    result = qa_stuff({"query": query})
    results.append(result['result']) 
    sources = result["source_documents"]
    contents = [source.page_content for source in sources]  

    processed_contents = []
    for content in contents:
        if isinstance(content, str):  
            processed_content = content.split('.')[:2]  
            processed_contents.extend(processed_content)  
        else:
            print(f"Expected a string but got {type(content)}") 
    contexts.append(processed_contents)

print(len(questions))
print(len(ground_truths))
print(len(results))
print(len(contexts))



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.
10
10
10
10


In [ ]:
# Creating a DataFrame
data = {
    'question': questions,
    'answer': results,
    'contexts': contexts,
    'ground_truth': ground_truths
}

df = pd.DataFrame(data)

# Display the DataFrame
print(df.head())

# Save as .csv
df.to_csv('ragas_aoai_dataset.csv', index=False)  

                                            question  \
0            Qu'est-ce que le service Azure OpenAI ?   
1       Comment fonctionne le service Azure OpenAI ?   
2  Quels sont les avantages du service Azure Open...   
3  Quels sont les cas d'utilisation du service Az...   
4  Comment mettre en œuvre le service Azure OpenAI ?   

                                              answer  \
0  Le service Azure OpenAI est un outil puissant ...   
1  Le service Azure OpenAI repose sur deux compos...   
2  Le texte mentionne qu'il y a plusieurs avantag...   
3  Le document ne fournit pas d'informations spéc...   
4  Pour mettre en œuvre le service Azure OpenAI, ...   

                                            contexts  \
0  [Le service Azure OpenAI : un outil puissant \...   
1  [Comment fonctionne le service Azure OpenAI ? ...   
2  [Quels sont les avantages du service Azure Ope...   
3  [Quels sont les avantages du service Azure Ope...   
4  [Comment mettre en œuvre le service Azure O

In [ ]:
# https://blog.cellenza.com/securite-2/utiliser-azure-openai-langchain-et-ragas-pour-la-classification-des-documents-confidentiels-et-la-protection-des-donnees-sensibles/ 

# from ragas.testset.generator import TestsetGenerator
# from ragas.testset.evolutions import simple, reasoning, multi_context

# testset = generator.generate_with_langchain_docs(documents, test_size=10)

# print(testset)

Evaluer


In [ ]:
# !pip install ragas datasets

In [ ]:
import pandas as pd 
from datasets import Dataset
from tqdm import tqdm
import pyarrow as py
import pyarrow as pa
import pyarrow.dataset as ds
from ragas import evaluate
from ragas.metrics import answer_relevancy, answer_similarity, answer_correctness
from ragas.metrics import context_recall, context_precision, context_entity_recall, context_utilization, context_relevancy
from ragas.metrics import faithfulness
from ragas.metrics.critique import harmfulness, maliciousness, coherence, correctness, conciseness
from ragas.llms import LangchainLLMWrapper
from datasets import Dataset, Features, Sequence, Value, load_dataset


# ragas.metrics.answer_relevancy : Scores the relevancy of the answer according to the given question.
# ragas.metrics.answer_similarity : Scores the semantic similarity of ground truth with generated answer.
# ragas.metrics.answer_correctness : Measures answer correctness compared to ground truth as a combination of factuality and semantic similarity.
# ragas.metrics.context_precision : Average Precision is a metric that evaluates whether all of the relevant items selected by the model are ranked higher or not.
# ragas.metrics.context_recall : Estimates context recall by estimating TP and FN using annotated answer and retrieved context.
# ragas.metrics.context_entity_recall : Calculates recall based on entities present in ground truth and context.

# Définir le dataset explicitement 
features = Features({
    "question": Value("string"),
    "answer": Value("string"),
    "contexts": Sequence(Value("string")),
    "ground_truth": Value("string")
})
dataset = Dataset.from_dict(df, features=features)

In [ ]:
from ragas.llms import LangchainLLMWrapper

# wrapper around azure_model 
ragas_azure_model = LangchainLLMWrapper(llm)
# patch the new RagasLLM instance
faithfulness.llm = ragas_azure_model
answer_relevancy.llm = ragas_azure_model
context_precision.llm = ragas_azure_model
context_recall.llm = ragas_azure_model
# harmfulness.llm = ragas_azure_model


In [ ]:
from langchain.embeddings import OpenAIEmbeddings

# only for answer_relevancy
azure_embeddings = OpenAIEmbeddings(
    # deployment="text-embedding-ada-002",
    model="text-embedding-ada-002",
    openai_api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    openai_api_type="azure"
)

answer_relevancy.embeddings = azure_embeddings
answer_similarity.embeddings = azure_embeddings
answer_correctness.embeddings = azure_embeddings

c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_community\embeddings\openai.py:336: UserWarning: If you have openai>=1.0.0 installed and are using Azure, please use the `AzureOpenAIEmbeddings` class.
  warnings.warn(


In [ ]:
evaluate_contexts = evaluate(
       dataset=dataset,
       metrics=[context_recall, context_precision],
       llm=ragas_azure_model,
       raise_exceptions=False,
)

evaluate_contexts

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 20/20 [00:07<00:00,  2.58it/s]


{'context_recall': 0.9000, 'context_precision': 0.8920}

In [ ]:
evaluate_contexts_df = evaluate_contexts.to_pandas()
evaluate_contexts_df

,question,answer,contexts,ground_truth,context_recall,context_precision
0,Qu'est-ce que le service Azure OpenAI ?,Le service Azure OpenAI est un outil puissant ...,[Le service Azure OpenAI : un outil puissant \...,"[""Le service Azure OpenAI est un service cloud...",1.0,1.000000
1,Comment fonctionne le service Azure OpenAI ?,Le service Azure OpenAI repose sur deux compos...,[Comment fonctionne le service Azure OpenAI ? ...,['Le service Azure OpenAI repose sur deux comp...,1.0,0.950000
2,Quels sont les avantages du service Azure Open...,Le texte mentionne qu'il y a plusieurs avantag...,[Quels sont les avantages du service Azure Ope...,['Le service Azure OpenAI présente plusieurs a...,1.0,0.876667
3,Quels sont les cas d'utilisation du service Az...,Le document ne fournit pas d'informations spéc...,[Quels sont les avantages du service Azure Ope...,"[""Le service Azure OpenAI peut être utilisé po...",1.0,0.710000
4,Comment mettre en œuvre le service Azure OpenAI ?,"Pour mettre en œuvre le service Azure OpenAI, ...",[Comment mettre en œuvre le service Azure Open...,['Pour mettre en œuvre le service Azure OpenAI...,0.5,0.450000
5,Quels sont les composants principaux du servic...,Le service Azure OpenAI repose sur deux compos...,[Comment fonctionne le service Azure OpenAI ? ...,['Le service Azure OpenAI repose sur deux comp...,1.0,1.000000
6,Quels sont les modèles d'IA accessibles via le...,"Unfortunately, I do not have enough informatio...","[modèle, en utilisant les outils de formation ...","[""Les API Azure OpenAI sont des interfaces de ...",1.0,0.966667
7,Quelles tâches les modèles d'IA du service Azu...,"Je suis désolé, mais les informations fournies...","[modèle, en utilisant les outils de formation ...","[""Il permet d'accéder à des modèles d'IA de po...",1.0,0.966667
8,Quelle est l'infrastructure sur laquelle repos...,Le texte ne donne pas d'informations sur l'inf...,[Comment fonctionne le service Azure OpenAI ? ...,"[""Il garantit une haute performance, en s'appu...",1.0,1.000000
9,Quels outils le service Azure OpenAI fournit-i...,Le service Azure OpenAI fournit des outils de ...,"[modèle, en utilisant les outils de formation ...","[""Choisir un modèle d'IA, parmi ceux proposés ...",0.5,1.000000


In [ ]:
evaluate_answers = evaluate(
       dataset=dataset,
       metrics=[faithfulness, answer_relevancy, answer_similarity, answer_correctness],
       llm=ragas_azure_model,
       raise_exceptions=False,
)

evaluate_answers

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\executor.py", line 78, in _aresults
    r = await future
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\asyncio\tasks.py", line 571, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\executor.py", line 37, in sema_coro
    return await coro
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\executor.py", line 111, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\metrics\base.py", line 125, in ascore
    raise e
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\s

Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\executor.py", line 78, in _aresults
    r = await future
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\asyncio\tasks.py", line 571, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\executor.py", line 37, in sema_coro
    return await coro
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\executor.py", line 111, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\metrics\base.py", line 125, in ascore
    raise e
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\metrics\base.py", line 121, in 

{'faithfulness': 0.8750, 'answer_relevancy': 1.0000, 'answer_similarity': 0.5000, 'answer_correctness': 1.0000}

In [ ]:
evaluate_answers_df = evaluate_contexts.to_pandas()
evaluate_answers_df

,question,answer,contexts,ground_truth,context_recall,context_precision
0,Qu'est-ce que le service Azure OpenAI ?,Le service Azure OpenAI est un outil puissant ...,[Le service Azure OpenAI : un outil puissant \...,"[""Le service Azure OpenAI est un service cloud...",1.0,1.000000
1,Comment fonctionne le service Azure OpenAI ?,Le service Azure OpenAI repose sur deux compos...,[Comment fonctionne le service Azure OpenAI ? ...,['Le service Azure OpenAI repose sur deux comp...,1.0,0.950000
2,Quels sont les avantages du service Azure Open...,Le texte mentionne qu'il y a plusieurs avantag...,[Quels sont les avantages du service Azure Ope...,['Le service Azure OpenAI présente plusieurs a...,1.0,0.876667
3,Quels sont les cas d'utilisation du service Az...,Le document ne fournit pas d'informations spéc...,[Quels sont les avantages du service Azure Ope...,"[""Le service Azure OpenAI peut être utilisé po...",1.0,0.710000
4,Comment mettre en œuvre le service Azure OpenAI ?,"Pour mettre en œuvre le service Azure OpenAI, ...",[Comment mettre en œuvre le service Azure Open...,['Pour mettre en œuvre le service Azure OpenAI...,0.5,0.450000
5,Quels sont les composants principaux du servic...,Le service Azure OpenAI repose sur deux compos...,[Comment fonctionne le service Azure OpenAI ? ...,['Le service Azure OpenAI repose sur deux comp...,1.0,1.000000
6,Quels sont les modèles d'IA accessibles via le...,"Unfortunately, I do not have enough informatio...","[modèle, en utilisant les outils de formation ...","[""Les API Azure OpenAI sont des interfaces de ...",1.0,0.966667
7,Quelles tâches les modèles d'IA du service Azu...,"Je suis désolé, mais les informations fournies...","[modèle, en utilisant les outils de formation ...","[""Il permet d'accéder à des modèles d'IA de po...",1.0,0.966667
8,Quelle est l'infrastructure sur laquelle repos...,Le texte ne donne pas d'informations sur l'inf...,[Comment fonctionne le service Azure OpenAI ? ...,"[""Il garantit une haute performance, en s'appu...",1.0,1.000000
9,Quels outils le service Azure OpenAI fournit-i...,Le service Azure OpenAI fournit des outils de ...,"[modèle, en utilisant les outils de formation ...","[""Choisir un modèle d'IA, parmi ceux proposés ...",0.5,1.000000


In [ ]:
evaluate_aspect_critique = evaluate(
       dataset=dataset,
       metrics=[harmfulness, maliciousness, coherence, correctness, conciseness],
       llm=ragas_azure_model,
       raise_exceptions=False,
)

evaluate_aspect_critique

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 50/50 [00:04<00:00, 10.74it/s]


{'harmfulness': 0.0000, 'maliciousness': 0.0000, 'coherence': 0.8000, 'correctness': 0.7000, 'conciseness': 0.8000}

In [ ]:
evaluate_aspect_critique_df = evaluate_aspect_critique.to_pandas()
evaluate_aspect_critique_df

,question,answer,contexts,ground_truth,harmfulness,maliciousness,coherence,correctness,conciseness
0,Qu'est-ce que le service Azure OpenAI ?,Le service Azure OpenAI est un outil puissant ...,[Le service Azure OpenAI : un outil puissant \...,"[""Le service Azure OpenAI est un service cloud...",0,0,1,1,1
1,Comment fonctionne le service Azure OpenAI ?,Le service Azure OpenAI repose sur deux compos...,[Comment fonctionne le service Azure OpenAI ? ...,['Le service Azure OpenAI repose sur deux comp...,0,0,1,1,1
2,Quels sont les avantages du service Azure Open...,Le texte mentionne qu'il y a plusieurs avantag...,[Quels sont les avantages du service Azure Ope...,['Le service Azure OpenAI présente plusieurs a...,0,0,1,0,0
3,Quels sont les cas d'utilisation du service Az...,Le document ne fournit pas d'informations spéc...,[Quels sont les avantages du service Azure Ope...,"[""Le service Azure OpenAI peut être utilisé po...",0,0,1,1,1
4,Comment mettre en œuvre le service Azure OpenAI ?,"Pour mettre en œuvre le service Azure OpenAI, ...",[Comment mettre en œuvre le service Azure Open...,['Pour mettre en œuvre le service Azure OpenAI...,0,0,1,1,1
5,Quels sont les composants principaux du servic...,Le service Azure OpenAI repose sur deux compos...,[Comment fonctionne le service Azure OpenAI ? ...,['Le service Azure OpenAI repose sur deux comp...,0,0,1,1,1
6,Quels sont les modèles d'IA accessibles via le...,"Unfortunately, I do not have enough informatio...","[modèle, en utilisant les outils de formation ...","[""Les API Azure OpenAI sont des interfaces de ...",0,0,1,1,1
7,Quelles tâches les modèles d'IA du service Azu...,"Je suis désolé, mais les informations fournies...","[modèle, en utilisant les outils de formation ...","[""Il permet d'accéder à des modèles d'IA de po...",0,0,0,0,0
8,Quelle est l'infrastructure sur laquelle repos...,Le texte ne donne pas d'informations sur l'inf...,[Comment fonctionne le service Azure OpenAI ? ...,"[""Il garantit une haute performance, en s'appu...",0,0,0,0,1
9,Quels outils le service Azure OpenAI fournit-i...,Le service Azure OpenAI fournit des outils de ...,"[modèle, en utilisant les outils de formation ...","[""Choisir un modèle d'IA, parmi ceux proposés ...",0,0,1,1,1


### BERT score

In [ ]:
#!pip install evaluate bert_score

Currently, the best model is microsoft/deberta-xlarge-mnli, please consider using it instead of the default roberta-large in order to have the best correlation with human evaluation.

https://docs.google.com/spreadsheets/d/1RKOVpselB98Nnh_EOC4A2BYn8_201tmPODpNWu4w7xI/edit#gid=0

In [ ]:
from evaluate import load

bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")
# default model lang=en: roberta-large
results

c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\paul.peton\.cache\huggingface\hub\models--roberta-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingfa

{'precision': [1.0, 1.0000001192092896],
 'recall': [1.0, 1.0000001192092896],
 'f1': [1.0, 1.0000001192092896],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.1)'}

precision: The precision for each sentence from the predictions + references lists, which ranges from 0.0 to 1.0.

recall: The recall for each sentence from the predictions + references lists, which ranges from 0.0 to 1.0.

f1: The F1 score for each sentence from the predictions + references lists, which ranges from 0.0 to 1.0.

hashcode: The hashcode of the library.

In [ ]:
# Ajuster les options pandas pour afficher les textes complets
pd.set_option('display.max_colwidth', None)  # Désactive la troncation des colonnes
pd.set_option('display.max_rows', None)      # Désactive la troncation des lignes
pd.set_option('display.max_columns', None)   # Affiche toutes les colonnes
pd.set_option('display.width', None)         # Ajuste automatiquement la largeur d'affichage


In [ ]:
predictions = answers
references = [sublist[0] for sublist in ground_truths]
results = bertscore.compute(predictions=predictions, references=references, lang="fr", model_type="microsoft/deberta-xlarge-mnli")
results

{'precision': [0.6400765180587769,
  0.6021711826324463,
  0.5712481141090393,
  0.8315932154655457,
  0.5764471888542175,
  0.8881711959838867,
  0.5767934322357178,
  0.6516508460044861,
  0.6587282419204712,
  0.6696093678474426],
 'recall': [0.8885854482650757,
  0.8658581972122192,
  0.8463621735572815,
  0.9536949992179871,
  0.7484399676322937,
  0.8556890487670898,
  0.6140142679214478,
  0.7516512274742126,
  0.6185110807418823,
  0.7025548219680786],
 'f1': [0.7441313862800598,
  0.710332989692688,
  0.6821095943450928,
  0.8884686231613159,
  0.6512798070907593,
  0.8716276288032532,
  0.594822108745575,
  0.6980879902839661,
  0.6379864811897278,
  0.6856865286827087],
 'hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.41.1)'}

In [ ]:
data = {
    'predictions': predictions,
    'references': references,
    'precision': results['precision'],
    'recall': results['recall'],
    'f1': results['f1']
}

# Créer le DataFrame
df = pd.DataFrame(data)

# Afficher le DataFrame
print(df.sort_values(by='precision', ascending=True).head(3))

                                                                                                                                                                                                                                                                                                               predictions  \
2  Le service Azure OpenAI présente plusieurs avantages, parmi lesquels l'accès à des modèles d'IA de pointe, une grande flexibilité, une haute performance grâce à l'infrastructure cloud de Microsoft Azure, et une facilité d'intégration grâce à des API simples et des SDK dans différents langages de programmation.   
4                  Pour mettre en œuvre le service Azure OpenAI, il faut créer un compte Microsoft Azure, souscrire à un abonnement au service, créer une clé d'API, choisir ou créer un modèle d'IA, créer un point de terminaison, envoyer des requêtes au point de terminaison, et analyser les résultats des requêtes.   
6                                             

In [ ]:
df.iloc[4]

predictions    Pour mettre en œuvre le service Azure OpenAI, il faut créer un compte Microsoft Azure, souscrire à un abonnement au service, créer une clé d'API, choisir ou créer un modèle d'IA, créer un point de terminaison, envoyer des requêtes au point de terminaison, et analyser les résultats des requêtes.
references                                                                                        [Pour mettre en œuvre le service Azure OpenAI, il faut suivre les étapes suivantes :, Analyser les résultats des requêtes, en utilisant les outils de suivi et de statistiques fournis par le portail Azure OpenAI.]
precision                                                                                                                                                                                                                                                                                                     0.577569
recall                                                             

In [ ]:
print(df.sort_values(by='precision', ascending=False).head(3))

                                                                                                                                                                                        predictions  \
5                                                                                           Les composants principaux du service Azure OpenAI sont le portail Azure OpenAI et les API Azure OpenAI.   
3  Le service Azure OpenAI peut être utilisé pour de nombreux cas d'utilisation, dans différents domaines et secteurs d'activité, tels que l'éducation, la santé, le divertissement et le commerce.   
9                                                                                          Le service Azure OpenAI fournit des outils de formation et de réglage pour créer son propre modèle d'IA.   

                                                                                                                       references  \
5                Le service Azure OpenAI repose sur deux composants pr

In [ ]:
df.iloc[5]

predictions             Les composants principaux du service Azure OpenAI sont le portail Azure OpenAI et les API Azure OpenAI.
references     Le service Azure OpenAI repose sur deux composants principaux : le portail Azure OpenAI et les API Azure OpenAI.
precision                                                                                                              0.888171
recall                                                                                                                 0.855689
f1                                                                                                                     0.871628
Name: 5, dtype: object